# 第六章 处理输入-链式 

在本章中，我们将学习如何通过使用链式 Prompt 将复杂任务拆分为一系列简单的子任务。

你可能会想，如果我们可以通过思维链推理一次性完成，那为什么要将任务拆分为多个 Prompt 呢？让我们用以下两个比喻来比较思维链推理和链式Prompt，从而来解释链式Prompt的必要性。 

- 将任务拆分为多个 Prompt 就像是一次性烹饪复杂菜肴与分阶段烹饪的区别。使用一个长而复杂的 Prompt 可能就像一次性烹饪复杂的菜肴，您必须同时管理多个成分、烹饪技巧和时间。这可能很具有挑战性，难以跟踪每个部分并确保每个组成部分都烹饪得恰到好处。另一方面，链式 Prompt 就像分阶段烹饪餐点，你专注于一个组成部分，确保每个部分都正确烹饪后再进行下一个。这种方法可以分解任务的复杂性，使其更易于管理，并减少错误的可能性。但是，对于非常简单的食谱，这种方法则可能因为过于复杂而不适用。

- 思维链推理和链式Prompt的区别就像一长串袋馍和简单的模块化程序。相比使用简单的模块化程序，一长串代码之前复杂的依赖关系会导致代码变得混乱且难以调试。

当你的工作流程包含多个步骤和多个状态，在不同状态下需要采取不同操作的时，链式 Prompt 就成为一种强大的策略。比如在对客户的查询进行分类后，获得查询的类别：是账户问题还是产品问题。然后可以根据不同的类别采取不同的行动。

在链式Prompt中，复杂任务被拆分为子任务，每个子任务仅包含执行对应任务所需的指令，这使得系统更易于管理，确保模型具备执行任务所需的所有信息，并降低了出错的可能性。此外，链式Prompt还有有以下好处
- 降低成本。因为更长的 Prompt 和更多的 tokens 会导致更高的运行成本，并且在某些情况下可能不需要概述所有步骤。
- 更容易测试哪些步骤可能更容易出错，或者在特定步骤中需要人工干预。
- 使用外部工具。例如，在其中的某一步，你可以去产品目录中查找某些内容，调用外部 API 或搜索知识库，这些使用单个 Prompt 无法实现的。

随着您与这些模型的构建和交互不断深入，您将逐渐培养出何时运用链式 Prompt策略的直觉。


## 一、使用链式 Prompt 实现产品查询

接下来，我们将使用链式 Prompt 来实现前面章节使用的案例 -- 回答顾客产品的查询，这次的产品列表将包含更多的产品。

In [1]:
%cd -q ../../../src/

In [2]:
import json 
from tool import get_completion_from_messages

### 1.1 提取相关产品和类别名称

In [3]:
delimiter = "####"

system_message = f"""
你将会被提供客服服务查询服务。
服务查询将使用{delimiter}字符分隔。

仅输出一个 Python 对象列表，其中每个对象具有以下格式：
    '类别': <计算机和笔记本电脑、智能手机和配件、电视和家庭影院系统、游戏机和配件、音频设备、相机和摄像机中的一个>,
和
    '产品': <产品必须在客户服务查询中找到，且必须在下面的允许产品列表中能找到的产品列表。如果没有找到，则输出空列表> 

其中类别和产品必须在客户服务查询中找到。
如果提及了产品，则必须将其与允许产品列表中的正确类别相关联。
如果在客户服务查询中未找到产品或类别，则输出空列表。

允许的产品列表：

计算机和笔记本电脑类别下的产品：
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

智能手机和配件类别下的产品：
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

电视和家庭影院系统类别下的产品：
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

游戏机和配件类别下的产品：
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

音频设备类别下的产品：
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

相机和摄像机类别下的产品：
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

仅输出 Python 对象列表，不包含其他字符信息。
"""

user_message_1 = f"""
 请查询 smartx prophone 智能手机和 fotosnap 相机，有dslr那款。
 另外，请查询关于电视产品的信息。 """

messages =  [{'role':'system', 'content': system_message},    
             {'role':'user', 'content': f"{delimiter}{user_message_1}{delimiter}"}] 

category_and_product_response_1 = get_completion_from_messages(messages)

print(category_and_product_response_1)

[{'类别': '智能手机和配件', '产品': ['SmartX ProPhone']}, {'类别': '相机和摄像机', '产品': ['FotoSnap DSLR Camera']}, {'类别': '电视和家庭影院系统', '产品': []}]


<br>

可以看到，输出是一个对象列表，每个对象都有一个类别和一些产品。如"SmartX ProPhone" 和 "Fotosnap DSLR Camera"。在最后一个对象中，只有类别，产品为空，因为用户查询没有提到任何具体的电视。

<br>

我们再来看一个例子。

In [4]:
user_message_2 = f"""我的路由器不工作了"""
messages =  [{'role':'system','content': system_message},
             {'role':'user','content': f"{delimiter}{user_message_2}{delimiter}"}] 
response = get_completion_from_messages(messages)
print(response)

{'类别': '计算机和笔记本电脑', '产品': []}


### 1.2 检索提取的产品和类别的详细信息

我们提供大量的产品信息作为示例，要求模型提取产品和对应的详细信息

In [5]:
products = {
    "TechPro Ultrabook": {
        "名称": "TechPro 超极本",
        "类别": "电脑和笔记本",
        "品牌": "TechPro",
        "型号": "TP-UB100",
        "保修期": "1 year",
        "评分": 4.5,
        "特色": ["13.3-inch display", "8GB RAM", "256GB SSD", "Intel Core i5 处理器"],
        "描述": "一款时尚轻便的超极本，适合日常使用。",
        "价格": 799.99
    },
    "BlueWave Gaming Laptop": {
        "名称": "BlueWave 游戏本",
        "类别": "电脑和笔记本",
        "品牌": "BlueWave",
        "型号": "BW-GL200",
        "保修期": "2 years",
        "评分": 4.7,
        "特色": ["15.6-inch display", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "描述": "一款高性能的游戏笔记本电脑，提供沉浸式体验。",
        "价格": 1199.99
    },
    "PowerLite Convertible": {
        "名称": "PowerLite Convertible",
        "类别": "电脑和笔记本",
        "品牌": "PowerLite",
        "型号": "PL-CV300",
        "保修期": "1 year",
        "评分": 4.3,
        "特色": ["14-inch touchscreen", "8GB RAM", "256GB SSD", "360-degree hinge"],
        "描述": "一款多功能的可转换笔记本电脑，具有灵敏的触摸屏。",
        "价格": 699.99
    },
    "TechPro Desktop": {
        "名称": "TechPro Desktop",
        "类别": "电脑和笔记本",
        "品牌": "TechPro",
        "型号": "TP-DT500",
        "保修期": "1 year",
        "评分": 4.4,
        "特色": ["Intel Core i7 processor", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "描述": "一款功能强大的台式电脑，适用于工作和娱乐。",
        "价格": 999.99
    },
    "BlueWave Chromebook": {
        "名称": "BlueWave Chromebook",
        "类别": "电脑和笔记本",
        "品牌": "BlueWave",
        "型号": "BW-CB100",
        "保修期": "1 year",
        "评分": 4.1,
        "特色": ["11.6-inch display", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "描述": "一款紧凑而价格实惠的Chromebook，适用于日常任务。",
        "价格": 249.99
    },
    "SmartX ProPhone": {
        "名称": "SmartX ProPhone",
        "类别": "智能手机和配件",
        "品牌": "SmartX",
        "型号": "SX-PP10",
        "保修期": "1 year",
        "评分": 4.6,
        "特色": ["6.1-inch display", "128GB storage", "12MP dual camera", "5G"],
        "描述": "一款拥有先进摄像功能的强大智能手机。",
        "价格": 899.99
    },
    "MobiTech PowerCase": {
        "名称": "MobiTech PowerCase",
        "类别": "专业手机",
        "品牌": "MobiTech",
        "型号": "MT-PC20",
        "保修期": "1 year",
        "评分": 4.3,
        "特色": ["5000mAh battery", "Wireless charging", "Compatible with SmartX ProPhone"],
        "描述": "一款带有内置电池的保护手机壳，可延长使用时间。",
        "价格": 59.99
    },
    "SmartX MiniPhone": {
        "名称": "SmartX MiniPhone",
        "类别": "专业手机",
        "品牌": "SmartX",
        "型号": "SX-MP5",
        "保修期": "1 year",
        "评分": 4.2,
        "特色": ["4.7-inch display", "64GB storage", "8MP camera", "4G"],
        "描述": "一款紧凑而价格实惠的智能手机，适用于基本任务。",
        "价格": 399.99
    },
    "MobiTech Wireless Charger": {
        "名称": "MobiTech Wireless Charger",
        "类别": "专业手机",
        "品牌": "MobiTech",
        "型号": "MT-WC10",
        "保修期": "1 year",
        "评分": 4.5,
        "特色": ["10W fast charging", "Qi-compatible", "LED indicator", "Compact design"],
        "描述": "一款方便的无线充电器，使工作区域整洁无杂物。",
        "价格": 29.99
    },
    "SmartX EarBuds": {
        "名称": "SmartX EarBuds",
        "类别": "专业手机",
        "品牌": "SmartX",
        "型号": "SX-EB20",
        "保修期": "1 year",
        "评分": 4.4,
        "特色": ["True wireless", "Bluetooth 5.0", "Touch controls", "24-hour battery life"],
        "描述": "通过这些舒适的耳塞体验真正的无线自由。",
        "价格": 99.99
    },

    "CineView 4K TV": {
        "名称": "CineView 4K TV",
        "类别": "电视和家庭影院系统",
        "品牌": "CineView",
        "型号": "CV-4K55",
        "保修期": "2 years",
        "评分": 4.8,
        "特色": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "描述": "一款色彩鲜艳、智能功能丰富的惊艳4K电视。",
        "价格": 599.99
    },
    "SoundMax Home Theater": {
        "名称": "SoundMax Home Theater",
        "类别": "电视和家庭影院系统",
        "品牌": "SoundMax",
        "型号": "SM-HT100",
        "保修期": "1 year",
        "评分": 4.4,
        "特色": ["5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth"],
        "描述": "一款强大的家庭影院系统，提供沉浸式音频体验。",
        "价格": 399.99
    },
    "CineView 8K TV": {
        "名称": "CineView 8K TV",
        "类别": "电视和家庭影院系统",
        "品牌": "CineView",
        "型号": "CV-8K65",
        "保修期": "2 years",
        "评分": 4.9,
        "特色": ["65-inch display", "8K resolution", "HDR", "Smart TV"],
        "描述": "通过这款惊艳的8K电视，体验未来。",
        "价格": 2999.99
    },
    "SoundMax Soundbar": {
        "名称": "SoundMax Soundbar",
        "类别": "电视和家庭影院系统",
        "品牌": "SoundMax",
        "型号": "SM-SB50",
        "保修期": "1 year",
        "评分": 4.3,
        "特色": ["2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth"],
        "描述": "使用这款时尚而功能强大的声音，升级您电视的音频体验。",
        "价格": 199.99
    },
    "CineView OLED TV": {
        "名称": "CineView OLED TV",
        "类别": "电视和家庭影院系统",
        "品牌": "CineView",
        "型号": "CV-OLED55",
        "保修期": "2 years",
        "评分": 4.7,
        "特色": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "描述": "通过这款OLED电视，体验真正的五彩斑斓。",
        "价格": 1499.99
    },

    "GameSphere X": {
        "名称": "GameSphere X",
        "类别": "游戏机和配件",
        "品牌": "GameSphere",
        "型号": "GS-X",
        "保修期": "1 year",
        "评分": 4.9,
        "特色": ["4K gaming", "1TB storage", "Backward compatibility", "Online multiplayer"],
        "描述": "一款下一代游戏机，提供终极游戏体验。",
        "价格": 499.99
    },
    "ProGamer Controller": {
        "名称": "ProGamer Controller",
        "类别": "游戏机和配件",
        "品牌": "ProGamer",
        "型号": "PG-C100",
        "保修期": "1 year",
        "评分": 4.2,
        "特色": ["Ergonomic design", "Customizable buttons", "Wireless", "Rechargeable battery"],
        "描述": "一款高品质的游戏手柄，提供精准和舒适的操作。",
        "价格": 59.99
    },
    "GameSphere Y": {
        "名称": "GameSphere Y",
        "类别": "游戏机和配件",
        "品牌": "GameSphere",
        "型号": "GS-Y",
        "保修期": "1 year",
        "评分": 4.8,
        "特色": ["4K gaming", "500GB storage", "Backward compatibility", "Online multiplayer"],
        "描述": "一款体积紧凑、性能强劲的游戏机。",
        "价格": 399.99
    },
    "ProGamer Racing Wheel": {
        "名称": "ProGamer Racing Wheel",
        "类别": "游戏机和配件",
        "品牌": "ProGamer",
        "型号": "PG-RW200",
        "保修期": "1 year",
        "评分": 4.5,
        "特色": ["Force feedback", "Adjustable pedals", "Paddle shifters", "Compatible with GameSphere X"],
        "描述": "使用这款逼真的赛车方向盘，提升您的赛车游戏体验。",
        "价格": 249.99
    },
    "GameSphere VR Headset": {
        "名称": "GameSphere VR Headset",
        "类别": "游戏机和配件",
        "品牌": "GameSphere",
        "型号": "GS-VR",
        "保修期": "1 year",
        "评分": 4.6,
        "特色": ["Immersive VR experience", "Built-in headphones", "Adjustable headband", "Compatible with GameSphere X"],
        "描述": "通过这款舒适的VR头戴设备，进入虚拟现实的世界。",
        "价格": 299.99
    },

    "AudioPhonic Noise-Canceling Headphones": {
        "名称": "AudioPhonic Noise-Canceling Headphones",
        "类别": "音频设备",
        "品牌": "AudioPhonic",
        "型号": "AP-NC100",
        "保修期": "1 year",
        "评分": 4.6,
        "特色": ["Active noise-canceling", "Bluetooth", "20-hour battery life", "Comfortable fit"],
        "描述": "通过这款降噪耳机，体验沉浸式的音效。",
        "价格": 199.99
    },
    "WaveSound Bluetooth Speaker": {
        "名称": "WaveSound Bluetooth Speaker",
        "类别": "音频设备",
        "品牌": "WaveSound",
        "型号": "WS-BS50",
        "保修期": "1 year",
        "评分": 4.5,
        "特色": ["Portable", "10-hour battery life", "Water-resistant", "Built-in microphone"],
        "描述": "一款紧凑而多用途的蓝牙音箱，适用于随时随地收听音乐。",
        "价格": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "名称": "AudioPhonic True Wireless Earbuds",
        "类别": "音频设备",
        "品牌": "AudioPhonic",
        "型号": "AP-TW20",
        "保修期": "1 year",
        "评分": 4.4,
        "特色": ["True wireless", "Bluetooth 5.0", "Touch controls", "18-hour battery life"],
        "描述": "通过这款舒适的真无线耳塞，无需线缆即可享受音乐。",
        "价格": 79.99
    },
    "WaveSound Soundbar": {
        "名称": "WaveSound Soundbar",
        "类别": "音频设备",
        "品牌": "WaveSound",
        "型号": "WS-SB40",
        "保修期": "1 year",
        "评分": 4.3,
        "特色": ["2.0 channel", "80W output", "Bluetooth", "Wall-mountable"],
        "描述": "使用这款纤薄而功能强大的声音吧，升级您电视的音频体验。",
        "价格": 99.99
    },
    "AudioPhonic Turntable": {
        "名称": "AudioPhonic Turntable",
        "类别": "音频设备",
        "品牌": "AudioPhonic",
        "型号": "AP-TT10",
        "保修期": "1 year",
        "评分": 4.2,
        "特色": ["3-speed", "Built-in speakers", "Bluetooth", "USB recording"],
        "描述": "通过这款现代化的唱片机，重拾您的黑胶唱片收藏。",
        "价格": 149.99
    },

    "FotoSnap DSLR Camera": {
        "名称": "FotoSnap DSLR Camera",
        "类别": "相机和摄像机",
        "品牌": "FotoSnap",
        "型号": "FS-DSLR200",
        "保修期": "1 year",
        "评分": 4.7,
        "特色": ["24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses"],
        "描述": "使用这款多功能的单反相机，捕捉惊艳的照片和视频。",
        "价格": 599.99
    },
    "ActionCam 4K": {
        "名称": "ActionCam 4K",
        "类别": "相机和摄像机",
        "品牌": "ActionCam",
        "型号": "AC-4K",
        "保修期": "1 year",
        "评分": 4.4,
        "特色": ["4K video", "Waterproof", "Image stabilization", "Wi-Fi"],
        "描述": "使用这款坚固而紧凑的4K运动相机，记录您的冒险旅程。",
        "价格": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "名称": "FotoSnap Mirrorless Camera",
        "类别": "相机和摄像机",
        "品牌": "FotoSnap",
        "型号": "FS-ML100",
        "保修期": "1 year",
        "评分": 4.6,
        "特色": ["20.1MP sensor", "4K video", "3-inch touchscreen", "Interchangeable lenses"],
        "描述": "一款具有先进功能的小巧轻便的无反相机。",
        "价格": 799.99
    },
    "ZoomMaster Camcorder": {
        "名称": "ZoomMaster Camcorder",
        "类别": "相机和摄像机",
        "品牌": "ZoomMaster",
        "型号": "ZM-CM50",
        "保修期": "1 year",
        "评分": 4.3,
        "特色": ["1080p video", "30x optical zoom", "3-inch LCD", "Image stabilization"],
        "描述": "使用这款易于使用的摄像机，捕捉生活的瞬间。",
        "价格": 249.99
    },
    "FotoSnap Instant Camera": {
        "名称": "FotoSnap Instant Camera",
        "类别": "相机和摄像机",
        "品牌": "FotoSnap",
        "型号": "FS-IC10",
        "保修期": "1 year",
        "评分": 4.1,
        "特色": ["Instant prints", "Built-in flash", "Selfie mirror", "Battery-powered"],
        "描述": "使用这款有趣且便携的即时相机，创造瞬间回忆。",
        "价格": 69.99
    }
}

In [6]:
def get_product_by_name(name):
    """
    根据产品名称获取产品

    参数:
    name: 产品名称
    """
    return products.get(name, None)

def get_products_by_category(category):
    """
    根据类别获取产品

    参数:
    category: 产品类别
    """
    return [product for product in products.values() if product["类别"] == category]

In [7]:
get_product_by_name("TechPro Ultrabook")

{'名称': 'TechPro 超极本',
 '类别': '电脑和笔记本',
 '品牌': 'TechPro',
 '型号': 'TP-UB100',
 '保修期': '1 year',
 '评分': 4.5,
 '特色': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'],
 '描述': '一款时尚轻便的超极本，适合日常使用。',
 '价格': 799.99}

In [8]:
get_products_by_category("电脑和笔记本")

[{'名称': 'TechPro 超极本',
  '类别': '电脑和笔记本',
  '品牌': 'TechPro',
  '型号': 'TP-UB100',
  '保修期': '1 year',
  '评分': 4.5,
  '特色': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'],
  '描述': '一款时尚轻便的超极本，适合日常使用。',
  '价格': 799.99},
 {'名称': 'BlueWave 游戏本',
  '类别': '电脑和笔记本',
  '品牌': 'BlueWave',
  '型号': 'BW-GL200',
  '保修期': '2 years',
  '评分': 4.7,
  '特色': ['15.6-inch display',
   '16GB RAM',
   '512GB SSD',
   'NVIDIA GeForce RTX 3060'],
  '描述': '一款高性能的游戏笔记本电脑，提供沉浸式体验。',
  '价格': 1199.99},
 {'名称': 'PowerLite Convertible',
  '类别': '电脑和笔记本',
  '品牌': 'PowerLite',
  '型号': 'PL-CV300',
  '保修期': '1 year',
  '评分': 4.3,
  '特色': ['14-inch touchscreen', '8GB RAM', '256GB SSD', '360-degree hinge'],
  '描述': '一款多功能的可转换笔记本电脑，具有灵敏的触摸屏。',
  '价格': 699.99},
 {'名称': 'TechPro Desktop',
  '类别': '电脑和笔记本',
  '品牌': 'TechPro',
  '型号': 'TP-DT500',
  '保修期': '1 year',
  '评分': 4.4,
  '特色': ['Intel Core i7 processor',
   '16GB RAM',
   '1TB HDD',
   'NVIDIA GeForce GTX 1660'],
  '描述': '一款功能强大的台式电脑，适用于工作和娱乐。',
  '价格':

#### 1.2.1 解析提取的产品和类别字符串

In [9]:
def read_string_to_list(input_string):
    """
    将输入的字符串转换为 Python 列表。

    参数:
    input_string: 输入的字符串，应为有效的 JSON 格式。

    返回:
    list 或 None: 如果输入字符串有效，则返回对应的 Python 列表，否则返回 None。
    """
    if input_string is None:
        return None

    try:
        # 将输入字符串中的单引号替换为双引号，以满足 JSON 格式的要求
        input_string = input_string.replace("'", "\"")  
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None   

In [10]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

[{'类别': '智能手机和配件', '产品': ['SmartX ProPhone']}, {'类别': '相机和摄像机', '产品': ['FotoSnap DSLR Camera']}, {'类别': '电视和家庭影院系统', '产品': []}]


#### 1.2.2 基于解析得到的产品和类别进行检索

In [11]:
def generate_output_string(data_list):
    """
    根据输入的数据列表生成包含产品或类别信息的字符串。

    参数:
    data_list: 包含字典的列表，每个字典都应包含 "products" 或 "category" 的键。

    返回:
    output_string: 包含产品或类别信息的字符串。
    """
    output_string = ""
    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "产品" in data and data["产品"]:
                products_list = data["产品"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "类别" in data:
                category_name = data["类别"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string 

In [12]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "名称": "SmartX ProPhone",
    "类别": "智能手机和配件",
    "品牌": "SmartX",
    "型号": "SX-PP10",
    "保修期": "1 year",
    "评分": 4.6,
    "特色": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "描述": "一款拥有先进摄像功能的强大智能手机。",
    "价格": 899.99
}
{
    "名称": "FotoSnap DSLR Camera",
    "类别": "相机和摄像机",
    "品牌": "FotoSnap",
    "型号": "FS-DSLR200",
    "保修期": "1 year",
    "评分": 4.7,
    "特色": [
        "24.2MP sensor",
        "1080p video",
        "3-inch LCD",
        "Interchangeable lenses"
    ],
    "描述": "使用这款多功能的单反相机，捕捉惊艳的照片和视频。",
    "价格": 599.99
}
{
    "名称": "CineView 4K TV",
    "类别": "电视和家庭影院系统",
    "品牌": "CineView",
    "型号": "CV-4K55",
    "保修期": "2 years",
    "评分": 4.8,
    "特色": [
        "55-inch display",
        "4K resolution",
        "HDR",
        "Smart TV"
    ],
    "描述": "一款色彩鲜艳、智能功能丰富的惊艳4K电视。",
    "价格": 599.99
}
{
    "名称": "SoundMax Home Theater",
    "类别": "电视和家庭影院系统",
    "品牌": "SoundMax",
    "型号": "SM-HT

### 1.3 根据详细的产品信息生成用户查询的答案

In [13]:
system_message = f"""
您是一家大型电子商店的客服助理。
请以友好和乐于助人的口吻回答问题，并尽量简洁明了。
请确保向用户提出相关的后续问题。
"""

user_message_1 = f"""
请查询 smartx prophone 智能手机和 fotosnap 相机，有dslr那款。
另外，介绍关于电视产品的信息。
"""

messages =  [{'role':'system','content': system_message},
             {'role':'user','content': user_message_1},  
             {'role':'assistant',
              'content': f"""相关产品信息:\n\
              {product_information_for_user_message_1}"""}]

final_response = get_completion_from_messages(messages)
print(final_response)

关于SmartX ProPhone智能手机和FotoSnap相机，我们有以下款式：

1. SmartX ProPhone SX-PP10智能手机：
   - 6.1英寸显示屏
   - 128GB存储空间
   - 12MP双摄像头
   - 支持5G网络
   - 价格为899.99美元

2. FotoSnap FS-DSLR200 DSLR相机：
   - 24.2MP传感器
   - 1080p视频拍摄
   - 3英寸液晶屏
   - 可更换镜头
   - 价格为599.99美元

关于电视产品，我们有以下几款可供选择：

1. CineView 4K电视 CV-4K55：
   - 55英寸显示屏
   - 4K分辨率
   - 支持HDR
   - 智能电视功能
   - 价格为599.99美元

2. CineView 8K电视 CV-8K65：
   - 65英寸显示屏
   - 8K分辨率
   - 支持HDR
   - 智能电视功能
   - 价格为2999.99美元

3. CineView OLED电视 CV-OLED55：
   - 55英寸显示屏
   - 4K分辨率
   - 支持HDR
   - 智能电视功能
   - 价格为1499.99美元

如果您对以上产品有任何进一步的问题或需要其他产品的信息，请随时告诉我。


### 1.4 总结

我们讨论了如何通过一系列步骤加载与用户查询相关的信息，为模型提供所需的上下文，以有效回答问题。

您可能会想，为什么我们选择性地将产品描述加载到提示中，而不是包含所有产品描述，让模型使用它所需的信息呢？

这其中有几个原因。

首先，包含过多的产品描述可能会使模型在处理上下文时感到困惑，就像对于试图一次处理大量信息的人一样。当然，对于像 GPT-4 这样更高级的模型来说，这个原因就不太重要了。尤其是当上下文像这个例子一样具有良好的结构时，模型足够聪明，能够巧妙地忽略那些明显不相关的信息。

接下来的原因更加具有说服力。

首先，包含所有产品描述可能会使模型对上下文更加混乱，就像对于试图一次处理大量信息的人一样。当然，对于像 GPT-4 这样更高级的模型来说，这个问题不太相关，特别是当上下文像这个例子一样结构良好时，模型足够聪明，只会忽略明显不相关的信息。接下来的原因更有说服力。

第二个原因是，语言模型有上下文限制，即固定数量的 token 允许作为输入和输出。如果您有一个巨大的产品目录，您甚至无法将所有描述都放入上下文窗口中。

最后一个原因是，包含所有产品描述可能会使模型过拟合，因为它会记住所有的产品描述，而不是只记住与查询相关的信息。这可能会导致模型在处理新的查询时表现不佳。

使用语言模型时，由于按 token 付费，可能会很昂贵。因此，通过有选择地加载信息，可以减少生成响应的成本。一般来说，确定何时动态加载信息到模型的上下文中，并允许模型决定何时需要更多信息，是增强这些模型能力的最佳方法之一。

并且要再次强调，您应该将语言模型视为需要必要上下文才能得出有用结论和执行有用任务的推理代理。因此，在这种情况下，我们必须向模型提供产品信息，然后它才能根据该产品信息进行推理，为用户创建有用的答案。

在这个例子中，我们只添加了一个特定函数或函数的调用，以通过产品名称获取产品描述或通过类别名称获取类别产品。但是，模型实际上擅长决定何时使用各种不同的工具，并可以正确地使用它们。这就是 ChatGPT 插件背后的思想。我们告诉模型它可以访问哪些工具以及它们的作用，它会在需要从特定来源获取信息或想要采取其他适当的操作时选择使用它们。在这个例子中，我们只能通过精确的产品和类别名称匹配查找信息，但还有更高级的信息检索技术。检索信息的最有效方法之一是使用自然语言处理技术，例如命名实体识别和关系提取。

另一方法是使用文本嵌入（Embedding）来获取信息。嵌入可以用于实现对大型语料库的高效知识检索，以查找与给定查询相关的信息。使用文本嵌入的一个关键优势是它们可以实现模糊或语义搜索，这使您能够在不使用精确关键字的情况下找到相关信息。因此，在此例子中，我们不一定需要产品的确切名称，而可以使用更一般的查询如 **“手机”** 进行搜索。我们计划很快推出一门全面的课程，介绍如何在各种应用中使用嵌入，敬请关注。

在下一章中我们将讨论如何评估语言模型的输出。

## 附录: 英文版提示

### 提取相关产品和类别名称

In [14]:
delimiter = "####"

system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a Python list of objects, where each object has \
the following format:
    'category': <one of Computers and Laptops, \
    Smartphones and Accessories, \
    Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, 
    Audio Equipment, Cameras and Camcorders>,
and
    'products': <products must be found in the customer service query. And products that must \
    be found in the allowed products below. If no products are found, output an empty list.
    >

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products: 

Products under Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Products under Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Products under Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Products under Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Products under Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Products under Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Only output the list of objects, with nothing else.
"""

user_message_1 = f"""
 tell me about the smartx pro phone and \
 the fotosnap camera, the dslr one. \
 Also tell me about your tvs """

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'CineView 8K TV', 'CineView OLED TV']}]


In [15]:
user_message_2 = f"""
my router isn't working"""
messages =  [  
{'role':'system',
 'content': system_message},    
{'role':'user',
 'content': f"{delimiter}{user_message_2}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

[]


### 检索提取的产品和类别的详细信息

In [16]:
products = {
    "TechPro Ultrabook": {
        "name": "TechPro Ultrabook",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-UB100",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["13.3-inch display", "8GB RAM", "256GB SSD", "Intel Core i5 processor"],
        "description": "A sleek and lightweight ultrabook for everyday use.",
        "price": 799.99
    },
    "BlueWave Gaming Laptop": {
        "name": "BlueWave Gaming Laptop",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-GL200",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["15.6-inch display", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "description": "A high-performance gaming laptop for an immersive experience.",
        "price": 1199.99
    },
    "PowerLite Convertible": {
        "name": "PowerLite Convertible",
        "category": "Computers and Laptops",
        "brand": "PowerLite",
        "model_number": "PL-CV300",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["14-inch touchscreen", "8GB RAM", "256GB SSD", "360-degree hinge"],
        "description": "A versatile convertible laptop with a responsive touchscreen.",
        "price": 699.99
    },
    "TechPro Desktop": {
        "name": "TechPro Desktop",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-DT500",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["Intel Core i7 processor", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "description": "A powerful desktop computer for work and play.",
        "price": 999.99
    },
    "BlueWave Chromebook": {
        "name": "BlueWave Chromebook",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-CB100",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["11.6-inch display", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "description": "A compact and affordable Chromebook for everyday tasks.",
        "price": 249.99
    },
    "SmartX ProPhone": {
        "name": "SmartX ProPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-PP10",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["6.1-inch display", "128GB storage", "12MP dual camera", "5G"],
        "description": "A powerful smartphone with advanced camera features.",
        "price": 899.99
    },
    "MobiTech PowerCase": {
        "name": "MobiTech PowerCase",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-PC20",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["5000mAh battery", "Wireless charging", "Compatible with SmartX ProPhone"],
        "description": "A protective case with built-in battery for extended usage.",
        "price": 59.99
    },
    "SmartX MiniPhone": {
        "name": "SmartX MiniPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-MP5",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["4.7-inch display", "64GB storage", "8MP camera", "4G"],
        "description": "A compact and affordable smartphone for basic tasks.",
        "price": 399.99
    },
    "MobiTech Wireless Charger": {
        "name": "MobiTech Wireless Charger",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-WC10",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["10W fast charging", "Qi-compatible", "LED indicator", "Compact design"],
        "description": "A convenient wireless charger for a clutter-free workspace.",
        "price": 29.99
    },
    "SmartX EarBuds": {
        "name": "SmartX EarBuds",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "24-hour battery life"],
        "description": "Experience true wireless freedom with these comfortable earbuds.",
        "price": 99.99
    },

    "CineView 4K TV": {
        "name": "CineView 4K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "2 years",
        "rating": 4.8,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "A stunning 4K TV with vibrant colors and smart features.",
        "price": 599.99
    },
    "SoundMax Home Theater": {
        "name": "SoundMax Home Theater",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-HT100",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth"],
        "description": "A powerful home theater system for an immersive audio experience.",
        "price": 399.99
    },
    "CineView 8K TV": {
        "name": "CineView 8K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "2 years",
        "rating": 4.9,
        "features": ["65-inch display", "8K resolution", "HDR", "Smart TV"],
        "description": "Experience the future of television with this stunning 8K TV.",
        "price": 2999.99
    },
    "SoundMax Soundbar": {
        "name": "SoundMax Soundbar",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-SB50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth"],
        "description": "Upgrade your TV's audio with this sleek and powerful soundbar.",
        "price": 199.99
    },
    "CineView OLED TV": {
        "name": "CineView OLED TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-OLED55",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "Experience true blacks and vibrant colors with this OLED TV.",
        "price": 1499.99
    },

    "GameSphere X": {
        "name": "GameSphere X",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-X",
        "warranty": "1 year",
        "rating": 4.9,
        "features": ["4K gaming", "1TB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A next-generation gaming console for the ultimate gaming experience.",
        "price": 499.99
    },
    "ProGamer Controller": {
        "name": "ProGamer Controller",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-C100",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["Ergonomic design", "Customizable buttons", "Wireless", "Rechargeable battery"],
        "description": "A high-quality gaming controller for precision and comfort.",
        "price": 59.99
    },
    "GameSphere Y": {
        "name": "GameSphere Y",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-Y",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["4K gaming", "500GB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A compact gaming console with powerful performance.",
        "price": 399.99
    },
    "ProGamer Racing Wheel": {
        "name": "ProGamer Racing Wheel",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-RW200",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Force feedback", "Adjustable pedals", "Paddle shifters", "Compatible with GameSphere X"],
        "description": "Enhance your racing games with this realistic racing wheel.",
        "price": 249.99
    },
    "GameSphere VR Headset": {
        "name": "GameSphere VR Headset",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-VR",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Immersive VR experience", "Built-in headphones", "Adjustable headband", "Compatible with GameSphere X"],
        "description": "Step into the world of virtual reality with this comfortable VR headset.",
        "price": 299.99
    },

    "AudioPhonic Noise-Canceling Headphones": {
        "name": "AudioPhonic Noise-Canceling Headphones",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-NC100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Active noise-canceling", "Bluetooth", "20-hour battery life", "Comfortable fit"],
        "description": "Experience immersive sound with these noise-canceling headphones.",
        "price": 199.99
    },
    "WaveSound Bluetooth Speaker": {
        "name": "WaveSound Bluetooth Speaker",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-BS50",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Portable", "10-hour battery life", "Water-resistant", "Built-in microphone"],
        "description": "A compact and versatile Bluetooth speaker for music on the go.",
        "price": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "name": "AudioPhonic True Wireless Earbuds",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TW20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "18-hour battery life"],
        "description": "Enjoy music without wires with these comfortable true wireless earbuds.",
        "price": 79.99
    },
    "WaveSound Soundbar": {
        "name": "WaveSound Soundbar",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-SB40",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.0 channel", "80W output", "Bluetooth", "Wall-mountable"],
        "description": "Upgrade your TV's audio with this slim and powerful soundbar.",
        "price": 99.99
    },
    "AudioPhonic Turntable": {
        "name": "AudioPhonic Turntable",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TT10",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["3-speed", "Built-in speakers", "Bluetooth", "USB recording"],
        "description": "Rediscover your vinyl collection with this modern turntable.",
        "price": 149.99
    },

    "FotoSnap DSLR Camera": {
        "name": "FotoSnap DSLR Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-DSLR200",
        "warranty": "1 year",
        "rating": 4.7,
        "features": ["24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses"],
        "description": "Capture stunning photos and videos with this versatile DSLR camera.",
        "price": 599.99
    },
    "ActionCam 4K": {
        "name": "ActionCam 4K",
        "category": "Cameras and Camcorders",
        "brand": "ActionCam",
        "model_number": "AC-4K",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["4K video", "Waterproof", "Image stabilization", "Wi-Fi"],
        "description": "Record your adventures with this rugged and compact 4K action camera.",
        "price": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "name": "FotoSnap Mirrorless Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-ML100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["20.1MP sensor", "4K video", "3-inch touchscreen", "Interchangeable lenses"],
        "description": "A compact and lightweight mirrorless camera with advanced features.",
        "price": 799.99
    },
    "ZoomMaster Camcorder": {
        "name": "ZoomMaster Camcorder",
        "category": "Cameras and Camcorders",
        "brand": "ZoomMaster",
        "model_number": "ZM-CM50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["1080p video", "30x optical zoom", "3-inch LCD", "Image stabilization"],
        "description": "Capture life's moments with this easy-to-use camcorder.",
        "price": 249.99
    },
    "FotoSnap Instant Camera": {
        "name": "FotoSnap Instant Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["Instant prints", "Built-in flash", "Selfie mirror", "Battery-powered"],
        "description": "Create instant memories with this fun and portable instant camera.",
        "price": 69.99
    }
}

In [17]:
def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

In [18]:
print(get_product_by_name("TechPro Ultrabook"))

{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 processor'], 'description': 'A sleek and lightweight ultrabook for everyday use.', 'price': 799.99}


In [19]:
print(get_products_by_category("Computers and Laptops"))

[{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 processor'], 'description': 'A sleek and lightweight ultrabook for everyday use.', 'price': 799.99}, {'name': 'BlueWave Gaming Laptop', 'category': 'Computers and Laptops', 'brand': 'BlueWave', 'model_number': 'BW-GL200', 'warranty': '2 years', 'rating': 4.7, 'features': ['15.6-inch display', '16GB RAM', '512GB SSD', 'NVIDIA GeForce RTX 3060'], 'description': 'A high-performance gaming laptop for an immersive experience.', 'price': 1199.99}, {'name': 'PowerLite Convertible', 'category': 'Computers and Laptops', 'brand': 'PowerLite', 'model_number': 'PL-CV300', 'warranty': '1 year', 'rating': 4.3, 'features': ['14-inch touchscreen', '8GB RAM', '256GB SSD', '360-degree hinge'], 'description': 'A versatile convertible laptop with a responsive touchscreen.', 'price': 

### 根据详细的产品信息生成用户查询的答案

In [20]:
system_message = f"""
You are a customer service assistant for a \
large electronic store. \
Respond in a friendly and helpful tone, \
with very concise answers. \
Make sure to ask the user relevant follow up questions.
"""
user_message_1 = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
messages =  [{'role':'system','content': system_message},   
             {'role':'user','content': user_message_1},
             {'role':'assistant',
              'content': f"""Relevant product information:\n\
              {product_information_for_user_message_1}"""}]
final_response = get_completion_from_messages(messages)
print(final_response)

The SmartX ProPhone is a powerful smartphone with advanced camera features. It has a 6.1-inch display, 128GB storage, a 12MP dual camera, and supports 5G. It is priced at $899.99.

The FotoSnap DSLR Camera is a versatile camera that allows you to capture stunning photos and videos. It features a 24.2MP sensor, 1080p video recording, a 3-inch LCD screen, and interchangeable lenses. It is priced at $599.99.

We have a range of TVs available. Some popular options include:

1. CineView 4K TV: It has a 55-inch display, 4K resolution, HDR, and is a smart TV. It is priced at $599.99.

2. CineView 8K TV: This stunning 8K TV offers a 65-inch display, 8K resolution, HDR, and is also a smart TV. It is priced at $2999.99.

3. CineView OLED TV: Experience vibrant colors with this OLED TV. It has a 55-inch display, 4K resolution, HDR, and is a smart TV. It is priced at $1499.99.

Please let me know if you have any specific questions or if there's anything else I can assist you with.
